<h1 style="color:blue">Segmenting and Clustering Neighborhoods in Toronto</h1>

<p>Start by importing the necessary libraries:</p>

In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

<p>Then use <b>BeautifulSoup</b> and <b>Pandas</b> to create the initial dataframe:</p>

In [27]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = pd.read_json(df[0].to_json(orient='records'))
df = df[['Postal code','Borough','Neighborhood']]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<p>Next, drop the rows where 'Borough' is 'Not assigned'. Please note that, contrary to what stated in the lab instructions, there are no rows where Borough is assigned and Neighborhood is not assigned. So the next code line is enough to clean the dataframe:<p>

In [28]:
df = df.drop(df[df.Borough == 'Not assigned'].index)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


<p>The source Wikipedia table column 'Neighborhood' has '/' to separate multiple Neighborhoods, instead of commas as required in the lab. Therefore replace them wherever they occur in that column:</p>

In [29]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' / ',', ')
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [31]:
df.shape

(103, 3)